## DISCOVERY EDA CORRELATIONS
- Calcular las correlaciones entre todas las series (correlaciones lineales que obligan que todas las series tengan el mismo tamaño)
- Se va a obtener una matriz gigante al 30k de series
- Poder saber qué series son más parecidas entre sí

In [1]:
import os
# set path root of repo
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('/')[:-1]
root_path = '/'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  /Users/joseortega/Documents/GitHub/forecasting-m5-dataset


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. Read raw files

In [41]:
folder_data_raw_dtype = 'data/data_input_dtype/'

df_calender = pd.read_pickle(folder_data_raw_dtype + 'calendar.pkl')
df_prices = pd.read_pickle(folder_data_raw_dtype + 'sell_prices.pkl')
df_sales = pd.read_pickle(folder_data_raw_dtype + 'sales_train_evaluation.pkl')
df_sample_output = pd.read_pickle(folder_data_raw_dtype + 'sample_submission.pkl')

In [4]:
df_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [5]:
df_sales.shape

(30490, 1947)

### 2. Armar df de la forma "id", "serie1", "serie2", ... ,"serieN". Para luego calcular correlaciones lineales

In [6]:
# eliminar columnas no utilizadas
aux_df_sales_delete_exogenas = df_sales.drop(columns = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])

# transponer y obtener data en formato columnas cada serie
data = aux_df_sales_delete_exogenas.T
data.columns = data.iloc[0]
data = data[1:]

# transformar todas las columnas que están en formato string a float
data[data.select_dtypes(include=['object']).columns] = data.select_dtypes(include=['object']).apply(pd.to_numeric, errors='coerce')

In [7]:
data

id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
d_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,0.0,...,0.0,14.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
d_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,...,0.0,11.0,1.0,0.0,4.0,0.0,0.0,6.0,0.0,0.0
d_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,...,0.0,5.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
d_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,6.0,1.0,0.0,5.0,2.0,0.0,2.0,0.0,0.0
d_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1937,0.0,0.0,0.0,1.0,0.0,0.0,1.0,5.0,0.0,1.0,...,3.0,6.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
d_1938,3.0,0.0,2.0,3.0,0.0,0.0,0.0,4.0,0.0,1.0,...,1.0,4.0,3.0,1.0,2.0,0.0,1.0,0.0,1.0,2.0
d_1939,3.0,0.0,3.0,0.0,2.0,5.0,1.0,1.0,0.0,0.0,...,3.0,4.0,3.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0
d_1940,0.0,0.0,0.0,2.0,1.0,2.0,1.0,40.0,1.0,0.0,...,0.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0,1.0,5.0


### 3. Calcular correlaciones lineales entre cada serie. Comparando el momento actual

In [8]:
%%time
data_corr = data.corr()

CPU times: user 33min 22s, sys: 30.9 s, total: 33min 53s
Wall time: 33min 58s


In [18]:
data_corr.head(3)

id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_evaluation,1.000000,0.037054,0.199233,0.186046,0.053664,-0.026536,0.058002,0.070329,-0.113826,-0.044841,...,0.168580,-0.074359,0.051665,0.201402,-0.062604,-0.017482,-0.133243,0.028761,0.152510,0.218534
HOBBIES_1_002_CA_1_evaluation,0.037054,1.000000,0.087505,0.002004,0.131606,0.042864,0.048103,0.011193,-0.057234,0.022361,...,0.078445,-0.077518,-0.001404,0.041396,0.097884,-0.068716,-0.048341,-0.067466,0.061103,0.045419
HOBBIES_1_003_CA_1_evaluation,0.199233,0.087505,1.000000,0.050581,0.067943,-0.012545,0.029045,0.087074,-0.115413,0.014040,...,0.115214,-0.066082,0.042739,0.225979,-0.069717,0.044204,-0.120748,-0.029837,0.194608,0.171492


In [9]:
### SE DEMORA DEMASIADO EN CALCULAR CORRELACIONES. MEJOR USAR OTRO METODO. MODELOS QUE ESTÉN OPTIMIZADOR/FUNCIONEN RAPIDO
# CON GRANDES VOLUMENES DE DATOS

#### GUARDAR

In [17]:
# Antes de optimizar
print("Tamaño original del DataFrame (en memoria):")
print(data_corr.memory_usage(deep=True).sum() / 1024**3, "GB")

Tamaño original del DataFrame (en memoria):
6.929777514189482 GB


In [19]:
# transformar columnas float64 a float32 para ahorrar espacio fisico de los 7gb que pesa la data
cols_float = data_corr.select_dtypes(include=['float64']).columns
data_corr[cols_float] = data_corr[cols_float].astype('float32')

In [22]:
# Luego de optimizar tipo de dato
print("Tamaño original del DataFrame (en memoria):")
print(data_corr.memory_usage(deep=True).sum() / 1024**3, "GB")

Tamaño original del DataFrame (en memoria):
3.466598268598318 GB


In [23]:
data_corr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30490 entries, HOBBIES_1_001_CA_1_evaluation to FOODS_3_827_WI_3_evaluation
Columns: 30490 entries, HOBBIES_1_001_CA_1_evaluation to FOODS_3_827_WI_3_evaluation
dtypes: float32(30490)
memory usage: 3.5+ GB


In [25]:
# guardar archivo con correlaciones cada serie formato pkl
path_folder_corr = 'data/corr_series/'
data_corr.to_pickle(path_folder_corr + 'corr_all_series.pkl', compression="gzip")

In [29]:
# revisar corr de una serie de ejemplo
data_corr['HOBBIES_1_001_CA_1_evaluation'].sort_values(ascending = False)

id
HOBBIES_1_001_CA_1_evaluation    1.000000
FOODS_3_499_WI_1_evaluation      0.385347
FOODS_3_192_CA_3_evaluation      0.383060
FOODS_3_389_TX_1_evaluation      0.378535
FOODS_3_561_CA_1_evaluation      0.376548
                                   ...   
FOODS_3_741_TX_2_evaluation     -0.271940
FOODS_3_093_TX_1_evaluation     -0.272903
FOODS_2_021_TX_2_evaluation     -0.283556
FOODS_3_080_WI_3_evaluation     -0.286920
FOODS_2_021_WI_3_evaluation     -0.290331
Name: HOBBIES_1_001_CA_1_evaluation, Length: 30490, dtype: float32

In [33]:
# revisar correlaciones altas
data_corr[data_corr >= 0.8]

id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_evaluation,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HOBBIES_1_002_CA_1_evaluation,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HOBBIES_1_003_CA_1_evaluation,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HOBBIES_1_004_CA_1_evaluation,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HOBBIES_1_005_CA_1_evaluation,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
FOODS_3_824_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
FOODS_3_825_WI_3_evaluation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [36]:
(data_corr >= 0.8).sum(axis = 0).sum()

30572

In [37]:
(data_corr >= 0.8).sum(axis = 1).sum()
# hay algunas pocas series que tienen correlación positiva mayor o igual a 0.8

30572